In [129]:
#solrpy library: http://pythonhosted.org/solrpy/overview.html
import solr #to install: pip install solrpy
#pandas library for data processing - only needed to index the solr core, can be removed otherwise
import pandas as pd #to install: pip install pandas
#scikit-optimize library: https://github.com/scikit-optimize
import skopt #to install: pip install scikit-optimize

# Settings

In [2]:
#Settings

# The files below are in the root folder of this GitHub repo. Launch jupyter notebook from that folder
# in order to read these files: 'jupyter notebook'

# Note: this is an artificial set of jobs, these are not real jobs, but are representative of our data
# Job descriptions are omitted, but usually we search that field also
jobs_data_file = "jobs.csv"

# File of relevancy judgements - these are highly subjective judgements, please don't take them too seriously
relevancy_file = "relevancy_judegements.csv"

#solr url and core (Jobs)
solr_url = "http://localhost:8983/solr/Jobs"

# Load Jobs Data, Index in Solr Jobs Core

In [3]:
# Note: You can skip this section if you were able to load the Solr Jobs Core along with the data directory from the 
# './Solr Core and Config' sub-folder. Older versions of Solr won't read this data, so here's some code to populate 
# the index from the jobs.csv file

jobs_df = pd.read_csv(jobs_data_file, sep=",")
jobs_df["jobSkills"] = jobs_df["jobSkills"].apply(lambda sk: sk.split("|"))
# assign a unique doc id to each row
jobs_df["id"] = range(len(jobs_df))
jobs_df.head(5)

,jobTitle,jobSkills,employer,city,state,geoCode,id
0,Lead Developer,"[Project management, Java, Programming, QA]",IT Services and Networking Corp.,New York,NY,"40.7127837,-74.0059413",0
1,Cloud Developer,"[QA, Software engineering, Compiler, Network, ...",Large Search Giant Llc.,Des Moines,IA,"41.6005448,-93.6091064",1
2,Application Developer,"[J2EE, Oracle, XML, QA, jQuery, JDBC, BIND, IBM]",Acme Inc,Des Moines,IA,"41.6005448,-93.6091064",2
3,Application Developer,"[Programming, Lifecycle management, Network]",IT Services and Networking Corp.,Chicago,IL,"41.8781136,-87.6297982",3
4,Pega Developer,"[QA, Agile, Architecture]",Scientists and Quants Inc,New York,NY,"40.7127837,-74.0059413",4


In [4]:
solr_connection = solr.Solr(solr_url, persistent=True, timeout=360, max_retries=5)

# convert dataframe to a list of dictionaries (required solr client library document format)
docs = jobs_df.T.to_dict().values()

#wipe out any existing documents if present
solr_connection.delete_query("*:*")

# send documents
solr_connection.add_many(docs)

# hard commit and optimize
solr_connection.commit()
solr_connection.optimize()

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n<lst name="responseHeader"><int name="status">0</int><int name="QTime">0</int></lst>\n</response>\n'

# Load Relevancy Judgements File

In [130]:
# The 'relevant' column is a list of document id's (the id field from the schema) that were both in the set of the top
# 20 returned documents, and were subjectively judged as relevant to the original
# query. We can subsequently use these to derive a MAP score for a given query

rel_df = pd.read_csv(relevancy_file, sep="|", converters={"fq": str, "location": str})
searches = rel_df.T.to_dict()
rel_df.head(3)

,query,fq,location,relevant
0,java developer,"{!geofilt}&sfield=geoCode&pt=41.884251,-87.632...","Chicago, IL","8,20,27,52,127,159,194,354,364,414,485,499,677..."
1,data warehouse,"{!geofilt}&sfield=geoCode&pt=41.884251,-87.632...","Chicago, IL","1078,1996,254,254,870,1968"
2,web services,"{!geofilt}&sfield=geoCode&pt=40.7127837, -74.0...","New York, NY","1342,1449,395,1272,1512,54,608,1528,38,84,150,..."


In [154]:
# Takes a search id and a qf setting, and returns the list of doc ids, 
def get_results_for_search(sid, qf_value, rows):
    search = searches[sid]
    fq = ""
    pt = "0,0"
    
    if not search["location"].strip() == "" :
        splt = filter(lambda s: "pt=" in s, search["fq"].split("&"))
        if splt:
            pt = splt[0].replace("pt=","")
            fq = "{!geofilt}"

    resp = solr_connection.select(
       q=search["query"], 
       fields="id",
       start=0, rows=rows, 
       qf=qf_value, # comes from get_solr_params
       fq=fq,
       sfield="geoCode",
       pt=pt,
       score=False,
       d="48.00", wt="json")
    predicted = list(map(lambda res: res["id"], resp.results))
    # return predicted doc ids, along with relevent ones (for IR metric)
    return predicted, list(map(int, search["relevant"].split(",")))

# IR Metrics

In [132]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : set
             A set of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mean_average_precision_at_k(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of sets of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

# Black Box Optimization

In [133]:
# Function takes a list of 12 real numbers, and returns a set of solr configuration options
def get_solr_params(params):
    return {"qf" : "employer^{0}  jobTitle^{1}  jobskills^{2}".format(*params[0:3])
            #"pf2" :  "employer^{0}  jobTitle^{1}  jobSkills^{2}".format(*params[3:6]), 
            #"pf"  :  "employer^{0}  jobTitle^{1}  jobSkills^{2}".format(*params[6:9]) 
           }

In [134]:
# spit into training and test set of queries
sids = list(searches.keys())
cutoff = len(sids) /2 
train_sids, test_sids = sids[:cutoff], sids[cutoff:]
train_sids, test_sids

([0, 1, 2, 3, 4], [5, 6, 7, 8, 9])

In [155]:
# Precision cut off
PREC_AT = 1000
# Black box objective function to minimize
def objective(params):
    # map list of numbers into solr parameters (just qf in this case)
    additional_params = get_solr_params(params)
    
    predicted, actual =[],[]
    for sid in train_sids:
        pred, act = get_results_for_search(sid, additional_params["qf"], PREC_AT)
        predicted.append(pred)
        actual.append(act)
    # Compute Mean average precision at 20
    return -1.0 * mean_average_precision_at_k(actual, predicted, PREC_AT)

# Run Optimizer Algorithm

In [156]:
# call back function to track progress
def callback(res):
    call_no = len(res.func_vals)
    current_fun = res.func_vals[-1]
    best = np.min(res.func_vals)
    MAP = -1.0 * best
    star = ""
    if best == current_fun:
        star = "*" * 4
    print str(call_no).ljust(5) + "\t" + str(MAP).ljust(20) + "\t" + \
        str(-1.0* current_fun).ljust(20) + "\t" + str(map(lambda d: round(d,3), res.x_iters[-1])) + "\t" + star

In [159]:
# Compute results at current production settings
prod_map = objective([1.8,1,1])
prod_map

-0.78419394166578438

In [ ]:
from skopt import gbrt_minimize
import datetime

ITERATIONS = 500 # probably want this to be high, 500 or more
min_val, max_val = 0.0, 50.0
space  = [(min_val, max_val) for i in range(3)] # min and max for each possible qf value

start = datetime.datetime.now()
print "Starting at ", start

print "Run","\t", "Best MAP", "\t\t", "Current MAP", "\t\t", "Parameters"
# run optimizer
res = gbrt_minimize(objective,       # the function to minimize
                  space,             # the bounds on each dimension of x
                  acq="LCB",         # controls how it searches for parameters
                  n_calls=ITERATIONS,# the number of evaluations of f including at x0
                  random_state=777,  # set to a fixed number if you want this to be deterministic
                  n_jobs=-1,         # how many threads (or really python processes due to GIL)
                  callback=callback) 

end = datetime.datetime.now()

Starting at  2016-10-12 18:02:55.976734
Run 	Best MAP 		Current MAP 		Parameters
1    	0.838371889286      	0.838371889286      	[7.633, 4.669, 11.146]	****
2    	0.838371889286      	0.609328774235      	[15.118, 3.984, 9.322]	
3    	0.838371889286      	0.838371889286      	[3.102, 29.481, 19.532]	****
4    	0.838371889286      	0.838371889286      	[22.993, 17.167, 9.658]	****
5    	0.838371889286      	0.838371889286      	[41.763, 49.444, 30.546]	****
6    	0.838371889286      	0.838371889286      	[46.35, 31.324, 44.14]	****
7    	0.838371889286      	0.838371889286      	[36.349, 34.089, 31.117]	****
8    	0.838371889286      	0.838371889286      	[38.425, 27.613, 12.656]	****
9    	0.838371889286      	0.838371889286      	[13.46, 13.443, 8.997]	****
10   	0.838371889286      	0.838371889286      	[32.201, 18.663, 40.82]	****
11   	0.838371889286      	0.838371889286      	[11.269, 15.497, 29.595]	****
12   	0.838371889286      	0.838371889286      	[20.547, 40.29, 35.926]	****

In [152]:
print("MAP @" + str(PREC_AT) + " PROD      = " +  str(-1 * prod_map))
print("MAP @" + str(PREC_AT) + " OPTIMIZED = " +  str(-1 * res.fun))
print("\nParameters:\n\t"),
print get_solr_params(res.x)["qf"]
print "\ngbrt_minimize took", (end - start).total_seconds(), "secs"

MAP @20 PROD      = 0.748947089947
MAP @20 OPTIMIZED = 0.834923613424

Parameters:
	employer^7.63318674507  jobTitle^4.66866284146  jobskills^11.1464049498

gbrt_minimize took 0.122591 secs
